In [ ]:
from __future__ import print_function
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.datasets import mnist

In [ ]:
# загрузка данных mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# преобразование данных в числа с плавающей точкой и нормализация

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train.reshape((-1, 784))        # или np.ravel
x_test = x_test.reshape((-1, 784))

In [ ]:
# создание функции для генерации мини_пакетов
def next_batch(batch_size):
    # получение случайного мини-пакета
    idx = tf.random.uniform([batch_size], 0, len(x_train), dtype=tf.int32)
    # извлечение данных и меток соответствующего мини-пакета
    x_batch = tf.gather(x_train, idx)
    y_batch = tf.gather(y_train, idx)
    return x_batch, y_batch

In [ ]:
# параметры обучения нейросети
learn_rate = 0.1
num_steps = 500
batch_size = 32
display_step = 100

In [ ]:
# конфигурация нейросети
n_hidden_1 = 256        # кол-во нейронов в 1 слое
n_hidden_2 = 256        # кол-во нейронов во 2 слое
num_input = 784         # MNIST-датасет = (размер изображжения 28х28)
num_classes = 10        # MNIST кол-во классов (цифры от 0 до 9)

In [ ]:
# входная часть графа
X = tf.placeholder('float', [None, num_input])
Y = tf.placeholder('float', [None, num_classes])

# определение весов и смещения
weights = {
    'h1': tf.Variable(tf.random.normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random.normal([n_hidden_2, num_classes]))
}

biases = {
    'h1': tf.Variable(tf.random.normal([n_hidden_1])),
    'h2': tf.Variable(tf.random.normal([n_hidden_2])),
    'out': tf.Variable(tf.random.normal([num_classes]))
}

In [ ]:
# создание модели
def neural_net(x):
    # первый слой, 256 нейронов
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['h1'])
    # второй слой, 256 нейронов
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['h2'])
    # выход полносвязанного слоя с нейроном на каждый класс
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [ ]:
# конструирование модели
logi = neural_net(X)
prediction = tf.nn.softmax(logi)

In [ ]:
# выбор loss-функции и optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logi, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learn_rate)
train_op = optimizer.minimize(loss_op)

In [ ]:
# оценка работы модели
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# инициализация переменных (назначение значений по умолчанию)
init = tf.global_variables_initializer()

In [ ]:
# старт процесса тренировки нейронной сети
with tf.Session() as sess:

    # запуск инициализатора
    sess.run(init)
    print('')
    for step in range(1, num_steps):
        batch_x, batch_y = next_batch(batch_size)
        # преобразование тензоров в значения
        batch_x, batch_y = sess.run([batch_x, batch_y])
        # преобразование пакета меток в one_hot форму (аналогично to_categorical в keras)
        # depth=10 - кол-во классов
        batch_y = tf.one_hot(batch_y, depth=10).eval()
        # запуск оптимизатора весов
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # вычисление батч ошибки и точности
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
        print(f'Step {step}, минибатч loss = {loss:.4f}, тренировочная точность = {acc:.4f}')
    print('Оптимизация завершена')

    # вычисление точности на изображениях датасета MNIST
    print(f'Тестовая точность: {sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels})}')
    